In [4]:
import math
import string
import random
import bisect

def function(x):
    return -x**2 + x + 2 
   
def random_generator(size = 6, chars = string.digits[:2]):
    """generates a random string with 0 and 1 of given size"""
    return ''.join(random.choice(chars) for x in range(size))

def b2to10(x):
    """converts a string from base 2 to base 10"""
    return int(x, 2)

def binaryToDomain(x10, b, a, l):
    """converts a binary string to a number in given interval [a,b]"""
    return x10 * (b - a) / (2 ** l - 1) + a

def crossover(s1, s2, p):
    """s1[p:], s2[p:] = s2[p:], s1[p:]"""
    s11 = list(s1)
    s22 = list(s2)
    s11[p:], s22[p:] = s22[p:], s11[p:]
    return ''.join(s11), ''.join(s22)

def mut(s):
    l = []
    for i in range(len(s)):
        if s[i] == '0':
            l.append('1')
        else:
            l.append('0')
    l2 = ""
    for x in l:
        l2 += x
    return l2

#dimensiune populatie
dim = 20
#domeniu functie
a = -1
b = 2
#precizie 
k = 6
#probabilitate crossover
pc = 0.25
#probabilitate mutatie
pm = 0.01
#numar etape algoritm
n = 50
#lungime cromozom
l = math.ceil(math.log((b - a) * (10 ** k), 2))

maxim = []
sumaM = 0
for k in range(n):
    c, x, f, p, s = [0] * dim, [0] * dim, [0] * dim, [0] * dim, 0
    for i in range(dim):
        c[i] = random_generator(l) #sir binar de lungime l
        x[i] = binaryToDomain(b2to10(c[i]), b, a, l) #sirul in baza 10 in [a,b]
        f[i] = function(x[i]) #f(x)
        
    if k==0:
        print("Populatia initiala")
    for i in range(dim):
        if k==0:
            print("cromozom ", i+1, ":", c[i], "x=", x[i], "f=", f[i])

    #suma de f(xi)
    for i in range(dim):
        s += f[i]

    #probabilitati selectie
    for i in range(dim):
        p[i] = f[i] / s

   
    if k==0: 
        print()
        print("Probabilitati selectie")
        for i in range(dim):
            print("cromozom", i+1, "probabilitate", p[i])

    #probabilitati cumulate
    q = [0] * dim
    for i in range(dim):
        q[i] = q[i-1] + p[i]
        
    if k==0:
        print()
        print("Intervale probabilitati selectie")
        print(q)
        print()

    #nr etape algoritm 
    u = [0] * dim
    crom_sel = [0] * dim
    for i in range(dim):
        u[i] = random.random() #numar aleator in [0,1) 
        poz = bisect.bisect_right(q, u[i]) 
        crom_sel[i] = c[poz] #selectez cromozomul
    #     print("u=", u[i], "din ", "[", q[poz-1],";", q[poz], "]")
        if k == 0:
            print("u=", u[i], "selectam cromozomul", poz)
#             print("u=", u[i], "selectam cromozomul", poz, "din ", "[", q[poz-1],";", q[poz], "]")
            
    if k == 0:
        print("Dupa selectie:")
        for i in range(dim):
            print(i+1, ":", crom_sel[i], "x=", binaryToDomain(b2to10(crom_sel[i]), b, a, l), "f=", function(binaryToDomain(b2to10(crom_sel[i]), b, a, l)))
        print()

        print("Probabilitatea de incrucisare:", pc)
    crom_crossover = [] 
    for i in range(dim):
        u[i] = random.random() #numar aleator in [0,1)
        if(u[i] < pc):
            if k == 0:
                print(i+1, ":", crom_sel[i], "u=", (u[i]), " < ", (pc), " participa")
            crom_crossover.append((i+1, crom_sel[i])) #pozitia + cromozom in sir binar
        else:
            if k == 0:
                print(i+1, ":", crom_sel[i], "u=", (u[i]))
    if k == 0:
        print()

    punct_rupere = []
    len_punct = len(crom_crossover)//2
    for i in range(len_punct):
        punct_rupere.append(random.randint(1, l))

    M = len(crom_crossover)
    m = len_punct
    new_crom_crossover = [None] * dim

    for i in range(dim):
        new_crom_crossover[i] = (i+1, crom_sel[i])

    #swap intre primul si ultimul, al 2lea si n-1, ... etc
    for i in range(m):
        s1, s2 = crossover(crom_crossover[i][1], crom_crossover[M - i - 1][1], punct_rupere[i])
        if k == 0:
            print("Recombinare dintre cromozomul", crom_crossover[i][0], "cu cromozomul", crom_crossover[M - i - 1][0] )
            print(crom_crossover[i][1], crom_crossover[M - i - 1][1], "punct", punct_rupere[i])
            print("Rezultat", s1, s2)
        new_crom_crossover[i] = (i+1, s1)
        new_crom_crossover[M - i - 1] = (M - i, s2)
    if k == 0:
        print()
    
    if k == 0:
        print("Dupa recombinare:")
        for i in range(dim):
            print(i+1, ":", new_crom_crossover[i], "x=", binaryToDomain(b2to10(new_crom_crossover[i][1]), b, a, l), "f=", function(binaryToDomain(b2to10(new_crom_crossover[i][1]), b, a, l)))
        print()    
        print("Probabilitate de mutatie pentru fiecare gena", pm)
#         print("Au fost modificati cromozomii:")

    mutatie = []
    # for i in range(dim):
    #     mutatie[i] = (i+1, new_crom_crossover[i])

    for i in range(dim):
        u[i] = random.random() #numar aleator in [0,1)
        if u[i] < pm:
            aux = mut(new_crom_crossover[i][1])
            mutatie.append((i+1, aux))
            if k == 0:
                print("Au fost modificati cromozomii:")
                print(i+1)
        else:
            mutatie.append((i+1, new_crom_crossover[i][1]))
            
    if k == 0:
        print()
        print("Dupa mutatie:")
        for i in range(dim):
            print(i+1, ":", mutatie[i], "x=", binaryToDomain(b2to10(mutatie[i][1]), b, a, l), "f=", function(binaryToDomain(b2to10(mutatie[i][1]), b, a, l))) 
            sumaM += function(binaryToDomain(b2to10(mutatie[i][1]), b, a, l))
        print()    

    lfun = []
    for i in range(dim):
        lfun.append(function(binaryToDomain(b2to10(mutatie[i][1]), b, a, l)))
    maxi = max(lfun)              
    maxim.append(maxi)
    
print("Evolutia maximului:")
suma = 0
for i in range(n):
    print(maxim[i])
    suma += maxim[i]
print("Valoarea medie a performantei:", suma/n, sumaM/n)

Populatia initiala
cromozom  1 : 0000010001011011010100 x= -0.9489450332987388 f= 0.15055829047891667
cromozom  2 : 1110100011001001101000 x= 1.727980310435369 f= 0.7420643571830547
cromozom  3 : 1100101001110101111011 x= 1.3725861007180455 f= 1.488593496833677
cromozom  4 : 0010010001001111000110 x= -0.5745042740116773 f= 1.0954405651306385
cromozom  5 : 0011011000110110011011 x= -0.3646961127987177 f= 1.5023006325107873
cromozom  6 : 0000100100011000010010 x= -0.8934197171735089 f= 0.3083814917920984
cromozom  7 : 0001000101100010010011 x= -0.7962815275863475 f= 0.5696542012384054
cromozom  8 : 0011000110101011011000 x= -0.4179361862984148 f= 1.4073931578839218
cromozom  9 : 1111101001111110001110 x= 1.9354660357155886 f= 0.18943726030697228
cromozom  10 : 0000011110010100011000 x= -0.9111766603414202 f= 0.258580433307636
cromozom  11 : 0100111000100000110101 x= -0.08443452940810425 f= 1.9084362808355277
cromozom  12 : 1100110101100100001110 x= 1.4069319741563735 f= 1.427474394252823